In [ ]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 31.5 MB/s 


In [ ]:
import matplotlib.pyplot as plt
import skimage
import math
import numpy as np
import tensorflow as tf
import math
import keras.backend as K
from datetime import datetime
import fractions
import itertools
import tqdm
from keras.utils.conv_utils import normalize_tuple
import os
import scipy
from datetime import datetime
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
import scipy.io

now = datetime.now() # current date and time
CURR_DATE = now.strftime("%m-%d-%Y")
MODEL_NAME = "CARE"+CURR_DATE

DATA_PATH = '/npz/Cervix_all_data_original.npz'
LABEL_PATH = '/npz/Cervix_all_data_labels_original.npz'
MODEL_PATH = '/CAREstd/checkpoint/weights.132-1.20_n1t1d3.hdf5'
MAIN_PATH= r"/objective_transfer/deep_learning" # Artem's Drive

In [ ]:
from google.colab import drive
import sys
import os
drive.mount('/content/drive')

In [ ]:
# Weights initializer for the layers.
kernel_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
# Gamma initializer for instance normalization.
gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
PATCH_SIZE = 512
input_img_size = (PATCH_SIZE, PATCH_SIZE,1)


# cycleGAN
"""
## Building blocks used in the CycleGAN generators and discriminators
"""


class ReflectionPadding2D(layers.Layer):
    """Implements Reflection Padding as a layer.
    Args:
        padding(tuple): Amount of padding for the
        spatial dimensions.
    Returns:
        A padded tensor with the same type as the input tensor.
    """

    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def call(self, input_tensor, mask=None):
        padding_width, padding_height = self.padding
        padding_tensor = [
            [0, 0],
            [padding_height, padding_height],
            [padding_width, padding_width],
            [0, 0],
        ]
        return tf.pad(input_tensor, padding_tensor, mode="REFLECT")


def residual_block(
    x,
    activation,
    kernel_initializer=kernel_init,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding="valid",
    gamma_initializer=gamma_init,
    use_bias=False,
):
    dim = x.shape[-1]
    input_tensor = x

    x = ReflectionPadding2D()(input_tensor)
    x = layers.Conv2D(
        dim,
        kernel_size,
        strides=strides,
        kernel_initializer=kernel_initializer,
        padding=padding,
        use_bias=use_bias,
    )(x)
    x = tfa.layers.InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    x = activation(x)

    x = ReflectionPadding2D()(x)
    x = layers.Conv2D(
        dim,
        kernel_size,
        strides=strides,
        kernel_initializer=kernel_initializer,
        padding=padding,
        use_bias=use_bias,
    )(x)
    x = tfa.layers.InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    x = layers.add([input_tensor, x])
    return x


def downsample(
    x,
    filters,
    activation,
    kernel_initializer=kernel_init,
    kernel_size=(3, 3),
    strides=(2, 2),
    padding="same",
    gamma_initializer=gamma_init,
    use_bias=False,
):
    x = layers.Conv2D(
        filters,
        kernel_size,
        strides=strides,
        kernel_initializer=kernel_initializer,
        padding=padding,
        use_bias=use_bias,
    )(x)
    x = tfa.layers.InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    if activation:
        x = activation(x)
    return x


def upsample(
    x,
    filters,
    activation,
    kernel_size=(3, 3),
    strides=(2, 2),
    padding="same",
    kernel_initializer=kernel_init,
    gamma_initializer=gamma_init,
    use_bias=False,
):
    x = layers.Conv2DTranspose(
        filters,
        kernel_size,
        strides=strides,
        padding=padding,
        kernel_initializer=kernel_initializer,
        use_bias=use_bias,
    )(x)
    x = tfa.layers.InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    if activation:
        x = activation(x)
    return x


"""
## Build the generators
The generator consists of downsampling blocks: nine residual blocks
and upsampling blocks. The structure of the generator is the following:
```
c7s1-64 ==> Conv block with `relu` activation, filter size of 7
d128 ====|
         |-> 2 downsampling blocks
d256 ====|
R256 ====|
R256     |
R256     |
R256     |
R256     |-> 9 residual blocks
R256     |
R256     |
R256     |
R256 ====|
u128 ====|
         |-> 2 upsampling blocks
u64  ====|
c7s1-3 => Last conv block with `tanh` activation, filter size of 7.
```
"""


def get_resnet_generator(
    filters=64,
    num_downsampling_blocks=2,
    num_residual_blocks=9,
    num_upsample_blocks=2,
    gamma_initializer=gamma_init,
    name=None,
):
    img_input = layers.Input(shape=input_img_size, name=name + "_img_input")
    x = ReflectionPadding2D(padding=(3, 3))(img_input)
    x = layers.Conv2D(filters, (7, 7), kernel_initializer=kernel_init, use_bias=False)(x)
    x = tfa.layers.InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    x = layers.Activation("relu")(x)

    # Downsampling
    for _ in range(num_downsampling_blocks):
        filters *= 2
        x = downsample(x, filters=filters, activation=layers.Activation("relu"))

    # Residual blocks
    for _ in range(num_residual_blocks):
        x = residual_block(x, activation=layers.Activation("relu"))

    # Upsampling
    for _ in range(num_upsample_blocks):
        filters //= 2
        x = upsample(x, filters, activation=layers.Activation("relu"))

    # Final block
    x = ReflectionPadding2D(padding=(3, 3))(x)
    x = layers.Conv2D(1, (7, 7), padding="valid")(x)
    x = layers.Activation("tanh")(x)

    model = keras.models.Model(img_input, x, name=name)
    return model


"""
## Build the discriminators
The discriminators implement the following architecture:
`C64->C128->C256->C512`
"""


def get_discriminator(
    filters=64, kernel_initializer=kernel_init, num_downsampling=3, name=None
):
    img_input = layers.Input(shape=input_img_size, name=name + "_img_input")
    x = layers.Conv2D(
        filters,
        (4, 4),
        strides=(2, 2),
        padding="same",
        kernel_initializer=kernel_initializer,
    )(img_input)
    x = layers.LeakyReLU(0.2)(x)

    num_filters = filters
    for num_downsample_block in range(3):
        num_filters *= 2
        if num_downsample_block < 2:
            x = downsample(
                x,
                filters=num_filters,
                activation=layers.LeakyReLU(0.2),
                kernel_size=(4, 4),
                strides=(2, 2),
            )
        else:
            x = downsample(
                x,
                filters=num_filters,
                activation=layers.LeakyReLU(0.2),
                kernel_size=(4, 4),
                strides=(1, 1),
            )

    x = layers.Conv2D(
        1, (4, 4), strides=(1, 1), padding="same", kernel_initializer=kernel_initializer
    )(x)

    model = keras.models.Model(inputs=img_input, outputs=x, name=name)
    return model


# Get the generators
gen_G = get_resnet_generator(name="generator_G")
gen_F = get_resnet_generator(name="generator_F")

# Get the discriminators
disc_X = get_discriminator(name="discriminator_X")
disc_Y = get_discriminator(name="discriminator_Y")


"""
## Build the CycleGAN model
We will override the `train_step()` method of the `Model` class
for training via `fit()`.
"""


class CycleGan(keras.Model):
    def __init__(
        self,
        generator_G,
        generator_F,
        discriminator_X,
        discriminator_Y,
        lambda_cycle=10.0,
        lambda_identity=0.5,
    ):
        super(CycleGan, self).__init__()
        self.gen_G = generator_G
        self.gen_F = generator_F
        self.disc_X = discriminator_X
        self.disc_Y = discriminator_Y
        self.lambda_cycle = lambda_cycle
        self.lambda_identity = lambda_identity

    def call(self, inputs):
        return (
            self.disc_X(inputs),
            self.disc_Y(inputs),
            self.gen_G(inputs),
            self.gen_F(inputs),
        )

    def compile(
        self,
        gen_G_optimizer,
        gen_F_optimizer,
        disc_X_optimizer,
        disc_Y_optimizer,
        gen_loss_fn,
        disc_loss_fn,
        metrics
    ):
        super(CycleGan, self).compile(metrics=metrics)
        self.gen_G_optimizer = gen_G_optimizer
        self.gen_F_optimizer = gen_F_optimizer
        self.disc_X_optimizer = disc_X_optimizer
        self.disc_Y_optimizer = disc_Y_optimizer
        self.generator_loss_fn = gen_loss_fn
        self.discriminator_loss_fn = disc_loss_fn
        self.cycle_loss_fn = keras.losses.MeanAbsoluteError()
        self.identity_loss_fn = keras.losses.MeanAbsoluteError() # TODO: disable

    def train_step(self, batch_data):
        # x is Horse and y is zebra
        # print("BATCH DATA: ", batch_data)
        real_x, real_y = batch_data

        # For CycleGAN, we need to calculate different
        # kinds of losses for the generators and discriminators.
        # We will perform the following steps here:
        #
        # 1. Pass real images through the generators and get the generated images
        # 2. Pass the generated images back to the generators to check if we
        #    we can predict the original image from the generated image.
        # 3. Do an identity mapping of the real images using the generators.
        # 4. Pass the generated images in 1) to the corresponding discriminators.
        # 5. Calculate the generators total loss (adverserial + cycle + identity)
        # 6. Calculate the discriminators loss
        # 7. Update the weights of the generators
        # 8. Update the weights of the discriminators
        # 9. Return the losses in a dictionary

        with tf.GradientTape(persistent=True) as tape:
            # print("1")
            # Horse to fake zebra
            fake_y = self.gen_G(real_x, training=True)
            # Zebra to fake horse -> y2x
            fake_x = self.gen_F(real_y, training=True)
            # Cycle (Horse to fake zebra to fake horse): x -> y -> x
            cycled_x = self.gen_F(fake_y, training=True)
            # Cycle (Zebra to fake horse to fake zebra) y -> x -> y
            cycled_y = self.gen_G(fake_x, training=True)
            # Identity mapping
            same_x = self.gen_F(real_x, training=True)
            same_y = self.gen_G(real_y, training=True)
            # print("4")
            # Discriminator output
            disc_real_x = self.disc_X(real_x, training=True)
            disc_fake_x = self.disc_X(fake_x, training=True)

            disc_real_y = self.disc_Y(real_y, training=True)
            disc_fake_y = self.disc_Y(fake_y, training=True)
            # print("5")
            # Generator adverserial loss
            gen_G_loss = self.generator_loss_fn(disc_fake_y)
            gen_F_loss = self.generator_loss_fn(disc_fake_x)
            # print("6")
            # Generator cycle loss
            cycle_loss_G = self.cycle_loss_fn(real_y, cycled_y) * self.lambda_cycle
            cycle_loss_F = self.cycle_loss_fn(real_x, cycled_x) * self.lambda_cycle
            # print("7")
            # Generator identity loss
            id_loss_G = (
                self.identity_loss_fn(real_y, same_y)
                * self.lambda_cycle
                * self.lambda_identity
            )
            id_loss_F = (
                self.identity_loss_fn(real_x, same_x)
                * self.lambda_cycle
                * self.lambda_identity
            )
            # print("8")
            # Total generator loss
            total_loss_G = gen_G_loss + cycle_loss_G + id_loss_G
            total_loss_F = gen_F_loss + cycle_loss_F + id_loss_F
            # print("9")
            # Discriminator loss
            disc_X_loss = self.discriminator_loss_fn(disc_real_x, disc_fake_x)
            disc_Y_loss = self.discriminator_loss_fn(disc_real_y, disc_fake_y)

        # Get the gradients for the generators
        grads_G = tape.gradient(total_loss_G, self.gen_G.trainable_variables)
        grads_F = tape.gradient(total_loss_F, self.gen_F.trainable_variables)
        # print("10")
        # Get the gradients for the discriminators
        disc_X_grads = tape.gradient(disc_X_loss, self.disc_X.trainable_variables)
        disc_Y_grads = tape.gradient(disc_Y_loss, self.disc_Y.trainable_variables)
        # print("11")
        # Update the weights of the generators
        self.gen_G_optimizer.apply_gradients(
            zip(grads_G, self.gen_G.trainable_variables)
        )
        self.gen_F_optimizer.apply_gradients(
            zip(grads_F, self.gen_F.trainable_variables)
        )
        # print("12")
        # Update the weights of the discriminators
        self.disc_X_optimizer.apply_gradients(
            zip(disc_X_grads, self.disc_X.trainable_variables)
        )
        self.disc_Y_optimizer.apply_gradients(
            zip(disc_Y_grads, self.disc_Y.trainable_variables)
        )

        return {
            "G_loss": total_loss_G,
            "F_loss": total_loss_F,
            "D_X_loss": disc_X_loss,
            "D_Y_loss": disc_Y_loss,
        }

# Loss function for evaluating adversarial loss
adv_loss_fn = keras.losses.MeanSquaredError()

def generator_loss_fn(fake):
  fake_loss = adv_loss_fn(tf.ones_like(fake), fake)
  return fake_loss


# Define the loss function for the discriminators
def discriminator_loss_fn(real, fake):
    real_loss = adv_loss_fn(tf.ones_like(real), real)
    fake_loss = adv_loss_fn(tf.zeros_like(fake), fake)
    return (real_loss + fake_loss) * 0.5

In [ ]:
def expandLastDim(data):
    return np.expand_dims(data, -1)

def normalizen1to1(img):
  # Normalizing images between 0 and 1 and preserving distribution
  img_norm = 2*(img - np.amin(img))/( np.amax(img)- np.amin(img)) - 1
  return img_norm

def load_data(path, expand=False, normalize=False):
    # Loading preprocessed image patches and adding 4th arbitrary dimension
    b = np.load(path)
    training_data = b['t']
    val_data = b['v']
    if expand == True:
      training_data = expandLastDim(training_data)
      val_data = expandLastDim(val_data)
    if normalize:
      training_data = [normalize0to1(m) for m in training_data]
      val_data = [normalize0to1(m) for m in val_data]
    return [training_data, val_data ]

def create_patches(img, patch_shape, slide):
    # returns stack of patches and number of patches
    patch_img = skimage.util.view_as_windows(img, (patch_shape,patch_shape), step=patch_shape-slide)
    patch = patch_img.reshape(patch_img.shape[0]*patch_img.shape[1],patch_shape,patch_shape) # more time efficient
    return patch

def patchify(input, patch_shape, slide):
    # getting number of input images
    len_to_allocate = int(np.shape(input)[0]*((slide-np.shape(input)[1]) / (slide-patch_shape))**2)
    data = np.zeros((len_to_allocate,patch_shape,patch_shape))
    count = 0
    # print("preparing to patchify: input, patch shape, slide",np.shape(input), patch_shape, slide)
    for i in range(np.shape(input)[0]):
      A = create_patches(input[i], patch_shape, slide)
      data[count:count+len(A),:,:] = A[:,:,:]
      count = count + len(A)
    print("      [PATCHIFYING COMPLETED] output shape, slide: ",np.shape(data),slide,"; number of images: ", np.shape(input)[0], ", number of patches: ", np.shape(data)[0])
    return data

def psnr(y_true, y_pred):
    '''
    Computs the peak signal-to-noise ratio between two images. Note that the
    maximum signal value is assumed to be 1.
    '''
    return tf.image.psnr(y_true, y_pred, max_val=1.0)

def ssim(y_true, y_pred):
    return tf.image.ssim(y_true, y_pred, 1, filter_size=3, filter_sigma=0.5, k2=0.05)

In [ ]:
def normalize0to1(img):
  # Normalizing images between 0 and 1 and preserving distribution
  if np.amin(img) == np.amax(img):
    print("ERROR, MAX AND MIN ARE EQUAL")
    return np.zeros_like(img)
  img_norm = (img - np.amin(img))/( np.amax(img)- np.amin(img))
  return img_norm

def mse(img1, img2):
  m = tf.keras.metrics.MeanSquaredError()
  m.update_state(img1, img2)
  return m.result().numpy()

def plot(img, label, hist=False, size=(10,10)):
  plt.figure(figsize = size)
  plt.axis('off')
  if hist==True:
    plt.hist(img.flatten(), bins=120)
    plt.axis('on')
  else: plt.imshow(img,cmap="gray")
  plt.title(label)

def plotThree(allThree, title, hist=False, save=False, savePath = None):
  # plot and saving merged images
  _, ax = plt.subplots(1, 3, figsize=(30,10))
  if hist:
    if np.shape(allThree[0])[0] > 1:
      raw = allThree[0].flatten()
      pred = allThree[1].flatten()
      gt = allThree[2].flatten()
    else:
      raw = allThree[0]
      pred = allThree[1]
      gt = allThree[2]
    print('raw | mean, median: ',np.mean(raw), np.median(raw))
    print('pred | mean, median: ',np.mean(pred), np.median(pred))
    print('gt | mean, median: ',np.mean(gt), np.median(gt))
    ax[0].hist(raw, bins=100)
    ax[1].hist(pred, bins=100)
    ax[2].hist(gt, bins=100)
  else:
    ax[0].imshow(allThree[0])
    ax[1].imshow(allThree[1])
    ax[2].imshow(allThree[2])
  ax[0].set_title("Input image")
  ax[1].set_title("Translated image")
  ax[2].set_title("GT image")
  # ax[0].axis("off")
  # ax[1].axis("off")
  # ax[2].axis("off")
  plt.show()

def getMode(x):
    values, counts = np.unique(x, return_counts=True)
    m = counts.argmax()
    # print("mode", m)
    return values[m]

def revert_img(img,original_size, patch_shape, slide):
  # reverts original image and removes overlaps by splitting overlap over 2 images
  step = int(patch_shape-slide)
  reconstructed_arr = np.zeros((original_size,original_size))
  for x in range(img.shape[0]):
    for y in range(img.shape[1]):
      start_x = int(slide/2)
      start_y = int(slide/2)
      end_x = 0
      end_y = 0
      if x == 0:
        start_x = 0
        end_x = int(slide/2)
      if y == 0:
        start_y = 0
        end_y = int(slide/2)
      if x == img.shape[0]-1: end_x = int(slide/2)
      if y == img.shape[1]-1: end_y = int(slide/2)
      x_pos, y_pos = x * step + start_x, y * step + start_y
      # print('x_pos, start_x: ',x_pos, start_x)
      # print('x_pos + step + end_x, start_x+step+end_x: ',x_pos + step + end_x, start_x+step+end_x)
      # print('y_pos, start_y: ',y_pos, start_y)
      # print('y_pos + step + end_y,  start_y+step+end_y: ',y_pos + step + end_y,  start_y+step+end_y)
      # print("end===============x,y: ",x,y)
      reconstructed_arr[x_pos : x_pos + step + end_x, y_pos : y_pos + step + end_y] = img[x, y, start_x:start_x+step+end_x, start_y:start_y+step+end_y]
  return reconstructed_arr

def merge_patches(img, original_size, patch_shape, slide):
  #  merging patches, img is a 3D array of stacked patches
  print("merging patches, img shape: ", img.shape, )
  row_len = int(math.sqrt(img.shape[0]))
  patches = np.zeros((row_len,row_len,patch_shape,patch_shape))
  print(img.shape)
  print(patches.shape)
  for r in range(row_len):
      patches[r,:,:,:] = img[r*row_len:r*row_len+row_len,:,:]
  # plt.figure(5)
  # plt.imshow(patches[1,1,:,:])
  return revert_img(patches,original_size,patch_shape, slide)

In [ ]:
# loading data
[training_data, val_data] = load_data(MAIN_PATH+DATA_PATH, expand=False, normalize=True) # change normalize property, CARE = false, CycleGAN = true
[training_data_labels,val_data_labels] = load_data(MAIN_PATH+LABEL_PATH, expand=False, normalize=True)

In [ ]:
# predicting
data_set = "val" # change this
patch = 128
slide = 64
if data_set == "train":
  input = training_data
  ground_truth = training_data_labels
else:
  input = val_data
  ground_truth = val_data_labels
if patch < 512:
    pretrain = patchify(input, patch, slide)
else: pretrain = input
pretrain, input, ground_truth = [expandLastDim(m) for m in [pretrain, input, ground_truth]]


In [ ]:
print("input",np.shape(pretrain))

In [ ]:
# loading CARE model
model = keras.models.load_model(
   MAIN_PATH+MODEL_PATH, custom_objects={"psnr": psnr,"ssim":ssim}, compile=False
)
prediction = model.predict(pretrain, batch_size = 1)

In [ ]:
def normalize_percentile(img, pmin=0.1, pmax=99.9, clip = False):
  eps=1e-20 # avoid zero division
  mi = np.percentile(img,pmin,axis=None,keepdims=True)
  ma = np.percentile(img,pmax,axis=None,keepdims=True)
  if clip == True: return np.clip((img - mi) / ( ma - mi + eps ), 0, 1)
  return (img - mi) / ( ma - mi + eps )

In [ ]:
val_ssim = []
val_psnr = []
val_mse = []
if True:
  for i in range(len(val_data)):
    gt = tf.convert_to_tensor([normalize0to1(ground_truth[i,...]),],dtype=tf.float32)
    # pred = tf.convert_to_tensor([normalize0to1(final_prediction[i,...]),],dtype=tf.float32)
    raw = tf.convert_to_tensor([normalize0to1(input[i,...]),],dtype=tf.float32)

    val_ssim.append(ssim(gt, raw).numpy()[0])
    val_psnr.append(psnr(gt, raw).numpy()[0])
    val_mse.append(mse(gt, raw))
    # print(i, "ssim, psnr, mse of gt vs pred: ",ssim(gt, pred).numpy()[0],psnr(gt, pred).numpy()[0],mse(gt, pred))
    print(i, "ssim, psnr, mse of gt vs raw: ",ssim(gt, raw).numpy()[0],psnr(gt, raw).numpy()[0],mse(gt, raw))
    # eval = [normalize0to1(raw.numpy()[0,192:321,192:321,0]), normalize0to1(pred.numpy()[0,192:321,192:321,0]), normalize0to1(gt.numpy()[0,192:321,192:321,0])]
    eval = [raw.numpy()[0,...,0], raw.numpy()[0,...,0], gt.numpy()[0,...,0]]
    # # eval = [(m) for m in eval]
    plotThree(eval, 'val, pred, gt', hist=False)
    plotThree(eval, 'val, pred, gt', hist=True)
    print("END====================================")
    print("")

  average_val_ssim = np.average(val_ssim)
  average_val_psnr = np.average(val_psnr)
  average_val_mse = np.average(val_mse)

In [ ]:
print(average_val_ssim,average_val_psnr,average_val_mse)
#print(np.shape(prediction))

0.26186514 17.286362 0.019016856


In [ ]:
# cycleGAN loading weights
model = CycleGan(
    generator_G=gen_G, generator_F=gen_F, discriminator_X=disc_X, discriminator_Y=disc_Y
)
model.built = True
model.load_weights(MAIN_PATH+MODEL_PATH)
prediction = model.gen_G.predict(pretrain, batch_size = 1)

In [ ]:
print(np.shape(prediction))

(34, 512, 512, 1)


In [ ]:
full_size_prediction = []
for i in range(np.shape(input)[0]):
  n = int((512-slide)/(patch-slide))**2
  # print("i, n, i*n, i*(n+1)",i, n, i*n, i*(n+1))
  full_size_prediction.append(merge_patches(prediction[i*n:(i+1)*(n),...,0], 512, patch, slide))

merging patches, img shape:  (1, 512, 512)
(1, 512, 512)
(1, 1, 512, 512)
merging patches, img shape:  (1, 512, 512)
(1, 512, 512)
(1, 1, 512, 512)
merging patches, img shape:  (1, 512, 512)
(1, 512, 512)
(1, 1, 512, 512)
merging patches, img shape:  (1, 512, 512)
(1, 512, 512)
(1, 1, 512, 512)
merging patches, img shape:  (1, 512, 512)
(1, 512, 512)
(1, 1, 512, 512)
merging patches, img shape:  (1, 512, 512)
(1, 512, 512)
(1, 1, 512, 512)
merging patches, img shape:  (1, 512, 512)
(1, 512, 512)
(1, 1, 512, 512)
merging patches, img shape:  (1, 512, 512)
(1, 512, 512)
(1, 1, 512, 512)
merging patches, img shape:  (1, 512, 512)
(1, 512, 512)
(1, 1, 512, 512)
merging patches, img shape:  (1, 512, 512)
(1, 512, 512)
(1, 1, 512, 512)
merging patches, img shape:  (1, 512, 512)
(1, 512, 512)
(1, 1, 512, 512)
merging patches, img shape:  (1, 512, 512)
(1, 512, 512)
(1, 1, 512, 512)
merging patches, img shape:  (1, 512, 512)
(1, 512, 512)
(1, 1, 512, 512)
merging patches, img shape:  (1, 512, 

In [ ]:
final_prediction = expandLastDim(full_size_prediction)
print(np.shape(final_prediction))

(34, 512, 512, 1)


In [ ]:
# TODO: as our data is normalized from -1 to 1, SSIM is quite off. Implement normalization 0 to 1 before calculating SSIM

# no shifting
val_ssim = []
val_psnr = []
val_mse = []
if True:
  for i in range(len(val_data)):
    gt = tf.convert_to_tensor([normalize0to1(ground_truth[i,...]),],dtype=tf.float32)
    pred = tf.convert_to_tensor([normalize0to1(final_prediction[i,...]),],dtype=tf.float32)
    raw = tf.convert_to_tensor([normalize0to1(input[i,...]),],dtype=tf.float32)

    val_ssim.append(ssim(gt, pred).numpy()[0])
    val_psnr.append(psnr(gt, pred).numpy()[0])
    val_mse.append(mse(gt, pred))
    print(i, "ssim, psnr, mse of gt vs pred: ",ssim(gt, pred).numpy()[0],psnr(gt, pred).numpy()[0],mse(gt, pred))
    print(i, "ssim, psnr, mse of gt vs raw: ",ssim(gt, raw).numpy()[0],psnr(gt, raw).numpy()[0],mse(gt, raw))
    # eval = [normalize0to1(raw.numpy()[0,192:321,192:321,0]), normalize0to1(pred.numpy()[0,192:321,192:321,0]), normalize0to1(gt.numpy()[0,192:321,192:321,0])]
    eval = [raw.numpy()[0,...,0], pred.numpy()[0,...,0], gt.numpy()[0,...,0]]
    # eval = [(m) for m in eval]
    plotThree(eval, 'val, pred, gt', hist=False)
    # plotThree(eval, 'val, pred, gt', hist=True)
    print("END====================================")
    print("")

  average_val_ssim = np.average(val_ssim)
  average_val_psnr = np.average(val_psnr)
  average_val_mse = np.average(val_mse)

In [ ]:
# # perform prediction and create stacks of result array of raw, pred, gt
# # TEST VERSION: shift mean of prediction to match gt mean, clip below 0
# val_ssim = []
# val_psnr = []
# val_mse = []
# if False:
#   for i in range(len(val_data)):
#     pred = final_prediction[i,...]
#     gt = ground_truth[i,...]
#     print(np.mean(pred),np.mean(gt))
#     mean_diff = np.mean(pred) - np.mean(gt)
#     print("mean_diff",mean_diff)
#     pred = pred - mean_diff
#     print("mode after sub", np.mean(pred),np.mean(gt))
#     pred = np.clip(pred, 0.0,1.0)
#     print("mode after clip", np.mean(pred),np.mean(gt))
#     gt = tf.convert_to_tensor([gt,],dtype=tf.float32)
#     pred = tf.convert_to_tensor([pred,],dtype=tf.float32)
#     raw = tf.convert_to_tensor([input[i,...],],dtype=tf.float32)

#     val_ssim.append(ssim(gt, pred).numpy()[0])
#     val_psnr.append(psnr(gt, pred).numpy()[0])
#     val_mse.append(mse(gt, pred))
#     print(i, "ssim, psnr, mse of gt vs pred: ",ssim(gt, pred).numpy()[0],psnr(gt, pred).numpy()[0],mse(gt, pred))
#     print(i, "ssim, psnr, mse of gt vs raw: ",ssim(gt, raw).numpy()[0],psnr(gt, raw).numpy()[0],mse(gt, raw))
#     eval = [raw.numpy()[0,...,0], pred.numpy()[0,...,0], gt.numpy()[0,...,0]]
#     # eval = [(m) for m in eval]
#     plotThree(eval, 'val, pred, gt', hist=False)
#     plotThree(eval, 'val, pred, gt', hist=True)
#     print("END====================================")
#     print("")

#   average_val_ssim = np.average(val_ssim)
#   average_val_psnr = np.average(val_psnr)
#   average_val_mse = np.average(val_mse)

In [ ]:
#validation
# stack_ranges = [[0,16],[16,34]]
# key_name = ["20210420_ROI3_512_shifted","20210421_ROI5_512_shifted"]

#train
# stack_ranges = [[0,21],[21,52],[52,73],[73,78],[78,86],[86,93],[93,110],[110,134],[134,143],[143,152],[152,180],[180,214],[214,238]]
# key_name = ["20210420_ROI2_512_shifted","20210421_ROI4_512_shifted","20210426_ROI1_512_shifted","20210426_ROI2_512_shifted","20210426_ROI3_512_shifted","20210426_ROI4_512_shifted","20210505_ROI1_512_shifted","20210505_ROI2_512_shifted","20210505_ROI3_512_shifted","20210505_ROI4_512_shifted","20210526_ROI3_512_shifted","20210615_ROI1_512_shifted","20210622_ROI4_512_shifted"]


In [ ]:
print(MODEL_NAME)

In [ ]:
#validation
stack_ranges = [[0,16],[16,34]]
key_name = ["20210420_ROI3_512_shifted","20210421_ROI5_512_shifted"]

# # saving validation stacks into .mat files
os.chdir(MAIN_PATH+'/validation/1009/')
for i in range(np.shape(stack_ranges)[0]):
    image_mat = []
    for n in range(stack_ranges[i][0],stack_ranges[i][1]):
        result = [input[n], final_prediction[n], ground_truth[n]]
        result = [normalize0to1(m) for m in result]
        result = np.stack(result)
        result = np.clip(255 * result, 0, 255).astype('uint8')
        image_mat.append([result[0],result[1],result[2]])
    scipy.io.savemat("NADH_"+MODEL_NAME + '_'+ key_name[i] +'_val.mat', {'images': image_mat})

In [ ]:

#train
stack_ranges = [[0,21],[21,52],[52,73],[73,78],[78,86],[86,93],[93,110],[110,134],[134,143],[143,152],[152,180],[180,214],[214,238]]
key_name = ["20210420_ROI2_512_shifted","20210421_ROI4_512_shifted","20210426_ROI1_512_shifted","20210426_ROI2_512_shifted","20210426_ROI3_512_shifted","20210426_ROI4_512_shifted","20210505_ROI1_512_shifted","20210505_ROI2_512_shifted","20210505_ROI3_512_shifted","20210505_ROI4_512_shifted","20210526_ROI3_512_shifted","20210615_ROI1_512_shifted","20210622_ROI4_512_shifted"]

# # saving validation stacks into .mat files
os.chdir(MAIN_PATH+'/train/1009/')
for i in range(np.shape(stack_ranges)[0]):
    image_mat = []
    for n in range(stack_ranges[i][0],stack_ranges[i][1]):
        result = [input[n], final_prediction[n], ground_truth[n]]
        result = [normalize0to1(m) for m in result]
        result = np.stack(result)
        result = np.clip(255 * result, 0, 255).astype('uint8')
        image_mat.append([result[0],result[1],result[2]])
    scipy.io.savemat("NADH_"+MODEL_NAME + '_'+ key_name[i] +'_val.mat', {'images': image_mat})